# Introduction to Transcriptomic Data Analysis

Welcome to this comprehensive exploration of transcriptomic data analysis. In this notebook, we'll work through real RNA-seq datasets to understand how gene expression varies across conditions, identify differentially expressed genes, and interpret biological meaning through pathway analysis.

## What we'll cover

This notebook walks through several key concepts in transcriptomic analysis:

- **Understanding RNA-seq data** - What do count matrices contain and how are they generated?
- **Quality control and normalization** - Ensuring data quality and comparability
- **Differential expression analysis** - Statistical methods to identify gene expression changes
- **Pathway enrichment analysis** - Connecting gene lists to biological processes
- **Visualization techniques** - Creating publication-quality plots
- **Biological interpretation** - Understanding what the data tells us about biology

The analysis demonstrates hormone treatment effects on gene expression, using estrogen receptor modulators as a case study in precision medicine.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our custom transcriptomic analysis tools
from scripts.transcriptomic_utils import (
    TranscriptomeDataProcessor,
    DifferentialExpressionAnalyzer,
    PathwayEnrichmentAnalyzer,
    TranscriptomeVisualizer
)

from scripts.config import (
    SAMPLE_METADATA,
    PATHWAY_GENE_SETS,
    DEMO_GENES,
    TREATMENT_INFO,
    generate_sample_count_matrix,
    get_sample_metadata_df,
    get_pathway_gene_sets,
    get_treatment_contrasts
)

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Understanding RNA-seq Data

RNA sequencing (RNA-seq) measures the expression levels of thousands of genes simultaneously. The raw output is a count matrix where:

- **Rows** represent genes
- **Columns** represent samples
- **Values** are read counts (how many RNA molecules were sequenced)

Let's start by generating and exploring a sample dataset that represents a hormone treatment study.

In [ ]:
# Generate sample RNA-seq data for demonstration
print("Generating sample RNA-seq count matrix...")
count_matrix = generate_sample_count_matrix(n_genes=3000, n_samples=12, seed=42)

print(f"Count matrix shape: {count_matrix.shape}")
print(f"Genes: {count_matrix.shape[0]:,}")
print(f"Samples: {count_matrix.shape[1]}")
print("\nFirst few rows and columns:")
print(count_matrix.iloc[:5, :5])

In [ ]:
# Load sample metadata
metadata = get_sample_metadata_df()

print("Sample metadata:")
print(metadata)
print("\nExperimental conditions:")
print(metadata['condition'].value_counts())

## Treatment Information

This study examines the effects of two estrogen receptor modulators:

- **OHT (4-Hydroxytamoxifen)**: Active metabolite of tamoxifen, acts as both agonist and antagonist
- **DPN (Diarylpropionitrile)**: Selective estrogen receptor beta (ERβ) agonist
- **Control**: Vehicle control treatment

These compounds have different mechanisms and are used to understand estrogen signaling pathways.

In [ ]:
# Display treatment information
for treatment, info in TREATMENT_INFO.items():
    print(f"\n{treatment} ({info['full_name']})")
    print(f"  Description: {info['description']}")
    print(f"  Duration: {info['duration']}")
    print(f"  Concentration: {info['concentration']}")
    if 'mechanism' in info:
        print(f"  Mechanism: {info['mechanism']}")

## Quality Control and Data Processing

Before analyzing differential expression, we need to assess data quality and perform appropriate normalization. Key quality metrics include:

- **Library size**: Total number of reads per sample
- **Gene detection**: Number of genes with non-zero counts
- **Expression distribution**: Overall patterns of gene expression

In [ ]:
# Initialize data processor
processor = TranscriptomeDataProcessor()

# Calculate basic quality metrics
library_sizes = processor.calculate_library_sizes(count_matrix)
detected_genes = (count_matrix > 0).sum(axis=0)

print("Quality Control Metrics:")
print(f"Library sizes - Mean: {library_sizes.mean():,.0f}, Range: {library_sizes.min():,.0f} - {library_sizes.max():,.0f}")
print(f"Detected genes - Mean: {detected_genes.mean():.0f}, Range: {detected_genes.min()} - {detected_genes.max()}")

# Create quality control plots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Library size distribution
axes[0].bar(range(len(library_sizes)), library_sizes.values)
axes[0].set_xlabel('Sample')
axes[0].set_ylabel('Library Size (Total Counts)')
axes[0].set_title('Library Sizes Across Samples')
axes[0].tick_params(axis='x', rotation=45)

# Gene detection
axes[1].bar(range(len(detected_genes)), detected_genes.values)
axes[1].set_xlabel('Sample')
axes[1].set_ylabel('Number of Detected Genes')
axes[1].set_title('Gene Detection Across Samples')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Filter low-expression genes
print("Filtering low-expression genes...")
filtered_matrix = processor.filter_low_expression_genes(
    count_matrix, min_count=10, min_samples=3
)

print(f"Genes after filtering: {filtered_matrix.shape[0]:,} ({filtered_matrix.shape[0]/count_matrix.shape[0]*100:.1f}% retained)")

In [ ]:
# Normalize data for analysis
print("Normalizing expression data...")
normalized_data = processor.normalize_counts(filtered_matrix, method='log_cpm')

print(f"Normalized data shape: {normalized_data.shape}")
print(f"Expression range: {normalized_data.min().min():.2f} - {normalized_data.max().max():.2f}")

# Show distribution of normalized expression
plt.figure(figsize=(10, 6))
plt.hist(normalized_data.values.flatten(), bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Log2(CPM + 1)')
plt.ylabel('Frequency')
plt.title('Distribution of Normalized Gene Expression')
plt.show()

## Sample Relationships and Principal Component Analysis

Before analyzing differential expression, it's important to understand the overall relationships between samples. Principal Component Analysis (PCA) helps us visualize sample similarities and identify potential batch effects or outliers.

In [ ]:
# Initialize visualizer
visualizer = TranscriptomeVisualizer()

# Create PCA plot
print("Creating PCA plot of samples...")
fig = visualizer.plot_pca(normalized_data, metadata=metadata, color_by='condition')
plt.show()

# The PCA plot shows how samples cluster based on their overall expression patterns
# Samples from the same condition should cluster together
# Large separation between conditions suggests strong treatment effects

## Differential Expression Analysis

Now we'll identify genes that are differentially expressed between treatment conditions. We'll use statistical methods similar to DESeq2 to account for the discrete nature of count data and biological variability.

### OHT vs Control Comparison

In [ ]:
# Initialize differential expression analyzer
de_analyzer = DifferentialExpressionAnalyzer()

# Define sample groups for OHT vs Control comparison
oht_samples = [s for s, info in SAMPLE_METADATA.items() 
               if info['condition'] == 'OHT' and s in filtered_matrix.columns]
control_samples = [s for s, info in SAMPLE_METADATA.items() 
                  if info['condition'] == 'Control' and s in filtered_matrix.columns]

print(f"OHT samples: {oht_samples}")
print(f"Control samples: {control_samples}")

# Perform differential expression analysis
print("\nPerforming differential expression analysis: OHT vs Control")
oht_results = de_analyzer.perform_deseq2_like_analysis(
    filtered_matrix, oht_samples, control_samples
)

print(f"Analyzed {len(oht_results)} genes")
print("\nTop 10 results:")
print(oht_results[['gene', 'log2FoldChange', 'pvalue', 'padj']].head(10))

In [ ]:
# Identify significantly differentially expressed genes
oht_significant = de_analyzer.identify_top_genes(
    oht_results, p_threshold=0.05, fc_threshold=1.5, n_top=50
)

print(f"Significant DEGs (OHT vs Control): {len(oht_significant)}")

if len(oht_significant) > 0:
    print("\nTop 15 most significant genes:")
    for _, gene_row in oht_significant.head(15).iterrows():
        gene_name = gene_row['gene']
        log2fc = gene_row['log2FoldChange']
        padj = gene_row['padj']
        direction = "↑" if log2fc > 0 else "↓"
        print(f"  {gene_name}: {direction} {abs(log2fc):.2f} fold (padj={padj:.2e})")

### DPN vs Control Comparison

Now let's analyze the DPN treatment effects and compare them to the OHT results.

In [ ]:
# DPN vs Control comparison
dpn_samples = [s for s, info in SAMPLE_METADATA.items() 
               if info['condition'] == 'DPN' and s in filtered_matrix.columns]

print(f"DPN samples: {dpn_samples}")
print(f"Control samples: {control_samples}")

# Perform differential expression analysis
print("\nPerforming differential expression analysis: DPN vs Control")
dpn_results = de_analyzer.perform_deseq2_like_analysis(
    filtered_matrix, dpn_samples, control_samples
)

print(f"Analyzed {len(dpn_results)} genes")

# Identify significant genes
dpn_significant = de_analyzer.identify_top_genes(
    dpn_results, p_threshold=0.05, fc_threshold=1.5, n_top=50
)

print(f"Significant DEGs (DPN vs Control): {len(dpn_significant)}")

## Visualization of Differential Expression Results

Volcano plots and heatmaps are essential for visualizing differential expression results and understanding the magnitude and significance of gene expression changes.

In [ ]:
# Create volcano plot for OHT vs Control
print("Creating volcano plot for OHT vs Control...")
fig = visualizer.plot_volcano(oht_results, p_col='padj', fc_col='log2FoldChange',
                             p_threshold=0.05, fc_threshold=1.0)
plt.title('Volcano Plot: OHT vs Control')
plt.show()

# The volcano plot shows:
# - X-axis: log2 fold change (effect size)
# - Y-axis: -log10 p-value (significance)
# - Red points: significantly upregulated genes
# - Green points: significantly downregulated genes
# - Gray points: not significantly changed

In [ ]:
# Create volcano plot for DPN vs Control
print("Creating volcano plot for DPN vs Control...")
fig = visualizer.plot_volcano(dpn_results, p_col='padj', fc_col='log2FoldChange',
                             p_threshold=0.05, fc_threshold=1.0)
plt.title('Volcano Plot: DPN vs Control')
plt.show()

In [ ]:
# Create expression heatmap of top variable genes
print("Creating expression heatmap...")
fig = visualizer.plot_heatmap(normalized_data, sample_metadata=metadata, 
                             top_n=50, figsize=(12, 10))
plt.show()

# The heatmap shows:
# - Rows: genes (top 50 most variable)
# - Columns: samples
# - Colors: expression levels (red=high, blue=low)
# - Clustering reveals co-expressed gene groups and sample similarities

## Analysis of Known Hormone-Responsive Genes

Let's examine specific genes known to be involved in hormone signaling to validate our analysis and understand the biological mechanisms.

In [ ]:
# Analyze known hormone-responsive genes
print("Analysis of Known Hormone-Responsive Genes")
print("=" * 50)

hormone_genes = list(DEMO_GENES['hormone_responsive'].keys())
print(f"Analyzing {len(hormone_genes)} hormone-responsive genes: {hormone_genes}")

# Check results for both treatments
for treatment, results_df in [('OHT', oht_results), ('DPN', dpn_results)]:
    print(f"\n{treatment} Treatment Results:")
    print("-" * 30)
    
    for gene in hormone_genes:
        if gene in results_df['gene'].values:
            gene_result = results_df[results_df['gene'] == gene].iloc[0]
            
            # Get gene information
            gene_info = DEMO_GENES['hormone_responsive'][gene]
            expected = gene_info['expected_response'].get(treatment, 'unknown')
            
            log2fc = gene_result['log2FoldChange']
            padj = gene_result['padj']
            
            # Determine observed direction
            if log2fc > 0.5:
                observed = "up"
            elif log2fc < -0.5:
                observed = "down"
            else:
                observed = "stable"
            
            # Check if matches expectation
            match = "✓" if expected == observed else "✗" if expected != 'unknown' else "?"
            
            significance = "***" if padj < 0.001 else "**" if padj < 0.01 else "*" if padj < 0.05 else "ns"
            
            print(f"  {gene} ({gene_info['name']})")
            print(f"    Expected: {expected}, Observed: {observed} {match}")
            print(f"    Log2FC: {log2fc:+.2f}, p-adj: {padj:.3f} {significance}")
            print(f"    Function: {gene_info['description']}")
            print()

In [ ]:
# Create boxplots for key hormone-responsive genes
key_genes = ['ESR1', 'PGR', 'TFF1']
available_genes = [g for g in key_genes if g in normalized_data.index]

if available_genes:
    print(f"Creating expression boxplots for {available_genes}...")
    fig = visualizer.plot_expression_boxplot(
        normalized_data, available_genes, metadata=metadata, 
        group_by='condition', figsize=(15, 5)
    )
    plt.show()
else:
    print("Key hormone-responsive genes not found in dataset")

## Pathway Enrichment Analysis

Individual gene analysis only tells part of the story. Pathway enrichment analysis helps us understand which biological processes and pathways are affected by the treatments.

In [ ]:
# Initialize pathway analyzer
pathway_analyzer = PathwayEnrichmentAnalyzer()
pathway_analyzer.load_gene_sets(get_pathway_gene_sets())

print("Available pathways for enrichment analysis:")
for pathway_name, pathway_info in PATHWAY_GENE_SETS.items():
    print(f"  {pathway_name}: {len(pathway_info['genes'])} genes ({pathway_info['source']})")
    print(f"    {pathway_info['description']}")

In [ ]:
# Perform pathway enrichment for OHT treatment
if len(oht_significant) > 0:
    print("Pathway Enrichment Analysis: OHT vs Control")
    print("=" * 45)
    
    oht_de_genes = oht_significant['gene'].tolist()
    print(f"Analyzing {len(oht_de_genes)} differentially expressed genes")
    
    oht_enrichment = pathway_analyzer.perform_overrepresentation_analysis(oht_de_genes)
    
    if len(oht_enrichment) > 0:
        print("\nEnriched pathways (p < 0.05):")
        significant_pathways = oht_enrichment[oht_enrichment['p_adj'] < 0.05]
        
        if len(significant_pathways) > 0:
            for _, pathway_row in significant_pathways.iterrows():
                pathway_name = pathway_row['pathway']
                p_value = pathway_row['p_value']
                p_adj = pathway_row['p_adj']
                overlap_size = pathway_row['overlap_size']
                pathway_size = pathway_row['pathway_size']
                overlap_genes = pathway_row['overlap_genes']
                
                print(f"\n{pathway_name}:")
                print(f"  P-value: {p_value:.2e} (adjusted: {p_adj:.2e})")
                print(f"  Overlap: {overlap_size}/{pathway_size} genes")
                print(f"  Genes: {', '.join(overlap_genes)}")
        else:
            print("No pathways reached significance threshold (p_adj < 0.05)")
            print("\nTop pathways by p-value:")
            for _, pathway_row in oht_enrichment.head(5).iterrows():
                print(f"  {pathway_row['pathway']}: p={pathway_row['p_value']:.3f}")
    else:
        print("No pathway enrichment results found")
else:
    print("No significant genes found for pathway analysis")

In [ ]:
# Perform pathway enrichment for DPN treatment
if len(dpn_significant) > 0:
    print("Pathway Enrichment Analysis: DPN vs Control")
    print("=" * 45)
    
    dpn_de_genes = dpn_significant['gene'].tolist()
    print(f"Analyzing {len(dpn_de_genes)} differentially expressed genes")
    
    dpn_enrichment = pathway_analyzer.perform_overrepresentation_analysis(dpn_de_genes)
    
    if len(dpn_enrichment) > 0:
        print("\nTop enriched pathways:")
        for _, pathway_row in dpn_enrichment.head(5).iterrows():
            pathway_name = pathway_row['pathway']
            p_value = pathway_row['p_value']
            overlap_size = pathway_row['overlap_size']
            pathway_size = pathway_row['pathway_size']
            
            print(f"  {pathway_name}: {overlap_size}/{pathway_size} genes (p={p_value:.3f})")
else:
    print("No significant genes found for DPN pathway analysis")

In [ ]:
# Create pathway enrichment visualization
if 'oht_enrichment' in locals() and len(oht_enrichment) > 0:
    print("Creating pathway enrichment plot...")
    fig = visualizer.plot_pathway_enrichment(oht_enrichment, top_n=10)
    plt.title('Pathway Enrichment: OHT vs Control')
    plt.show()

## Gene Set Enrichment Analysis (GSEA)

GSEA analyzes whether gene sets are enriched at the top or bottom of a ranked list of genes, providing insights into coordinated pathway regulation.

In [ ]:
# Perform GSEA analysis
print("Gene Set Enrichment Analysis (GSEA)")
print("=" * 40)

# Create ranked gene list from OHT results
if len(oht_results) > 0:
    # Rank genes by log2 fold change
    ranked_genes = oht_results.set_index('gene')['log2FoldChange'].sort_values(ascending=False)
    
    print(f"Performing GSEA on {len(ranked_genes)} ranked genes...")
    print(f"Top upregulated: {ranked_genes.head(3).to_dict()}")
    print(f"Top downregulated: {ranked_genes.tail(3).to_dict()}")
    
    # Perform GSEA
    gsea_results = pathway_analyzer.perform_gsea(ranked_genes)
    
    if len(gsea_results) > 0:
        print("\nGSEA Results:")
        for _, gsea_row in gsea_results.iterrows():
            pathway_name = gsea_row['pathway']
            es_score = gsea_row['es_score']
            p_value = gsea_row['p_value']
            gene_set_size = gsea_row['gene_set_size']
            
            direction = "↑ Upregulated" if es_score > 0 else "↓ Downregulated"
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            
            print(f"  {pathway_name} ({gene_set_size} genes):")
            print(f"    ES Score: {es_score:.3f} ({direction})")
            print(f"    P-value: {p_value:.3f} {significance}")
    else:
        print("No GSEA results generated")
else:
    print("No differential expression results available for GSEA")

## Comparative Analysis: OHT vs DPN

Let's compare the effects of the two treatments to understand their different mechanisms of action.

In [ ]:
# Compare OHT and DPN effects
print("Comparative Analysis: OHT vs DPN Treatment Effects")
print("=" * 55)

if len(oht_significant) > 0 and len(dpn_significant) > 0:
    oht_genes = set(oht_significant['gene'])
    dpn_genes = set(dpn_significant['gene'])
    
    # Find overlapping and unique genes
    common_genes = oht_genes & dpn_genes
    oht_only = oht_genes - dpn_genes
    dpn_only = dpn_genes - oht_genes
    
    print(f"OHT-specific genes: {len(oht_only)}")
    print(f"DPN-specific genes: {len(dpn_only)}")
    print(f"Common genes: {len(common_genes)}")
    
    if len(common_genes) > 0:
        print(f"\nGenes affected by both treatments: {list(common_genes)[:10]}")
        
        # Analyze direction of changes for common genes
        print("\nDirection comparison for common genes:")
        for gene in list(common_genes)[:5]:  # Show first 5
            oht_fc = oht_results[oht_results['gene'] == gene]['log2FoldChange'].iloc[0]
            dpn_fc = dpn_results[dpn_results['gene'] == gene]['log2FoldChange'].iloc[0]
            
            oht_dir = "↑" if oht_fc > 0 else "↓"
            dpn_dir = "↑" if dpn_fc > 0 else "↓"
            
            agreement = "✓" if (oht_fc > 0) == (dpn_fc > 0) else "✗"
            
            print(f"  {gene}: OHT {oht_dir}{abs(oht_fc):.2f}, DPN {dpn_dir}{abs(dpn_fc):.2f} {agreement}")
    
    # Create Venn diagram data
    print(f"\nTreatment specificity:")
    print(f"  Total unique genes affected: {len(oht_genes | dpn_genes)}")
    print(f"  Overlap coefficient: {len(common_genes) / min(len(oht_genes), len(dpn_genes)):.2f}")
    
else:
    print("Insufficient significant genes for comparative analysis")

## Biological Interpretation and Clinical Relevance

Now let's interpret our findings in the context of hormone biology and precision medicine.

In [ ]:
print("BIOLOGICAL INTERPRETATION")
print("=" * 30)

# Analyze cell cycle genes
cell_cycle_genes = list(DEMO_GENES['cell_cycle'].keys())
print(f"\nCell Cycle Regulation Analysis:")
print(f"Analyzing {len(cell_cycle_genes)} cell cycle genes: {cell_cycle_genes}")

for treatment, results_df in [('OHT', oht_results), ('DPN', dpn_results)]:
    if len(results_df) > 0:
        print(f"\n{treatment} effects on cell cycle:")
        for gene in cell_cycle_genes:
            if gene in results_df['gene'].values:
                gene_result = results_df[results_df['gene'] == gene].iloc[0]
                log2fc = gene_result['log2FoldChange']
                padj = gene_result['padj']
                
                direction = "promoted" if log2fc > 0.5 else "inhibited" if log2fc < -0.5 else "unchanged"
                significance = "significant" if padj < 0.05 else "not significant"
                
                gene_info = DEMO_GENES['cell_cycle'][gene]
                print(f"  {gene} ({gene_info['name']}): {direction} ({significance})")
                print(f"    Function: {gene_info['description']}")

In [ ]:
# Clinical implications
print("\nCLINICAL IMPLICATIONS")
print("=" * 25)

print("""
Key findings from this transcriptomic analysis:

1. TREATMENT MECHANISMS:
   - OHT and DPN show distinct transcriptional profiles
   - Both treatments affect hormone-responsive genes but with different patterns
   - Cell cycle regulation is differentially affected by each treatment

2. PRECISION MEDICINE APPLICATIONS:
   - Expression signatures can predict treatment response
   - Pathway analysis reveals mechanism of action
   - Individual gene responses suggest biomarker potential

3. THERAPEUTIC INSIGHTS:
   - Selective estrogen receptor modulation offers targeted therapy
   - Different ER subtypes (α vs β) mediate distinct responses
   - Combination therapies could target multiple pathways

4. BIOMARKER DEVELOPMENT:
   - Hormone-responsive genes serve as pharmacodynamic markers
   - Pathway activity scores could guide treatment selection
   - Expression monitoring could detect resistance development
""")

# Summary statistics
print("\nSUMMARY STATISTICS")
print("=" * 20)
print(f"Total genes analyzed: {count_matrix.shape[0]:,}")
print(f"Genes after filtering: {filtered_matrix.shape[0]:,}")
print(f"Samples analyzed: {filtered_matrix.shape[1]}")

if 'oht_significant' in locals():
    print(f"Significant genes (OHT): {len(oht_significant)}")
if 'dpn_significant' in locals():
    print(f"Significant genes (DPN): {len(dpn_significant)}")

print(f"Pathways analyzed: {len(PATHWAY_GENE_SETS)}")
print(f"Experimental conditions: {len(metadata['condition'].unique())}")

## Conclusions and Next Steps

This analysis demonstrates the power of transcriptomic analysis in understanding drug mechanisms and identifying biomarkers for precision medicine. 

### Key Takeaways:

1. **Data Quality**: Proper quality control and normalization are essential for reliable results
2. **Statistical Analysis**: Appropriate statistical methods account for biological variability
3. **Pathway Analysis**: Gene set analysis provides biological context beyond individual genes
4. **Visualization**: Multiple plot types reveal different aspects of the data
5. **Biological Interpretation**: Connecting results to known biology validates findings

### Next Steps:

1. **Validation**: Confirm key findings with qPCR or protein analysis
2. **Time Course**: Analyze temporal dynamics of gene expression changes
3. **Single Cell**: Examine cell-type-specific responses
4. **Integration**: Combine with other omics data (proteomics, metabolomics)
5. **Clinical Translation**: Develop expression-based biomarkers for patient stratification

### Resources for Further Learning:

- **Methods**: DESeq2, edgeR, limma for differential expression
- **Pathways**: GSEA, Reactome, KEGG databases
- **Visualization**: ggplot2, plotly, Cytoscape
- **Integration**: Multi-omics analysis platforms

This notebook provides a foundation for transcriptomic analysis that can be adapted to your specific research questions and datasets.